# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin

import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rishimadhav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rishimadhav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rishimadhav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("labeledmessages", con=engine)
X = df['message']
y = df.drop(columns=['id', 'message', 'original', 'genre'], axis=1)

In [4]:
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [6]:
def tokenize(text):
    """
    Receives text data and processes it
    Args: Text Data (column of messages)
    Returns: Tokenized text
    """
    # get list of URLS using regex
    detected_urls = re.findall(url_regex, text)

    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    # tokenize text
    tokens = word_tokenize(text)

    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token and lemmatize, normalize, remove leading/trailing white space and return clean tokenized text
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#def multi_tester(X,y):
#    '''
#    Function to create and test multiple ML algorithms and models
#    Args: training data of X and y
#    Returns: list of selected fitted models 
#    '''
#    pipe_1 = Pipeline() 

#pipeline = Pipeline([
#                    ('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer()),
#                    ('clf', MultiOutputClassifier(RandomForestClassifier()))
#                    ])

In [9]:
# Building a ML pipeline with other classification algorithms

model_pipeline = []
model_pipeline.append(LogisticRegression())
model_pipeline.append(SVC())
#model_pipeline.append(AdaBoostClassifier())
#model_pipeline.append(ExtraTreesClassifier())
model_pipeline.append(GaussianNB())
model_pipeline.append(RandomForestClassifier())
#model_pipeline.append(SGDClassifier())
#model_pipeline.append(GradientBoostingClassifier())
#model_pipeline.append(ExtraTreesClassifier())

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the pipeline
#pipeline.fit(X_train, y_train)

model_list=['Logistic Regression', 'SVM', 'Ada Boost', 'Extra Trees', 'Naive Bayes', ' Random Forest', 'SGD', 'Gradient Boost']
acc_list = []
auc_list = []
cm_list = []

In [11]:
for model in model_pipeline:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_list.append(metrics.accuracy_score(y_test, y_pred))
    fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
    auc_list.append(round(metrics.auc(fpr, tpr),2))
    cm_list.append(confusion_matrix(y_test, y_pred))

ValueError: could not convert string to float: 'The failure of rescuers to reach these areas promptly has forced some residents to dig out corpses with their bare hands, while mechanical diggers plough through the rubble in Padang, a city of 900,000 people.'

In [ ]:
#plot the confusion matrix to visualize it as a heatmap
fig = plt.figure(figsize=(18,10))
for i in range(len(cm_list)):
    cm = cm_list[i]
    model = model_list[i]
    sub = fig.add_subplot(2,3,i+1).set_title(model)
    cm_plot = sns.heatmap(cm, annot=True, cmap='Blues_r')
    cm_plot.set_xlabel('Predicted Values')
    cm_plot.set_ylabel('Actual Values')


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
# Test the model and out the Classification Report for each of the 36 categories of messages
def perf_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

perf_model(pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.76      0.27      0.39      1247
           1       0.81      0.97      0.88      3959

    accuracy                           0.80      5206
   macro avg       0.78      0.62      0.64      5206
weighted avg       0.80      0.80      0.77      5206

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4304
           1       0.88      0.46      0.60       902

    accuracy                           0.90      5206
   macro avg       0.89      0.72      0.77      5206
weighted avg       0.89      0.90      0.88      5206

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      520

/Users/rishimadhav/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishimadhav/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishimadhav/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Building the pipeline


parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.